# A tronbot using neat

In [1]:
import neat

In [2]:
from tronlib import *

In [3]:
# Create an evaluation function


from __future__ import print_function
import os
import neat
import visualize
import numpy as np

# If changing these, update config as well
BOARD_WIDTH = 10
BOARD_HEIGHT = 10



def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = evaluate_single_genome(genome, config)
            
def evaluate_single_genome(genome, config, n_games=3):
    score = 0
    # Play n games
    bot2 = NNBot(genome, config)
    for i in range(n_games):
        board = TronBoard(BOARD_WIDTH, BOARD_HEIGHT, (3, 4), (5, 4), (1, 0), (1, 0))
        bot1 = SimpleBot()
        game = TronGame(board, bot1, bot2, show=False)
        
        for step, state in enumerate(game):
            pass  # Game updated
        # Winning is a big deal
        if game.board.winner == 2:
            score += 10000
        
        # More score for lasting longer
        score += step
        
    return score

class NNBot:
    
    def __init__(self, genome, config):
        self.net = neat.nn.FeedForwardNetwork.create(genome, config)
    
    def decide_move(self, board, player_position):
        move_choice = np.argmax(self.net.activate(board.board.flatten()))
        return TronBoard.VALID_VECTORS[move_choice]

In [4]:
%%writefile tron.config

#--- parameters for the XOR-2 experiment ---#

[NEAT]
fitness_criterion     = max
fitness_threshold     = 500000
pop_size              = 150
reset_on_extinction   = False

[DefaultGenome]
# node activation options
activation_default      = sigmoid
activation_mutate_rate  = 0.0
activation_options      = sigmoid

# node aggregation options
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum

# node bias options
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# genome compatibility options
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5

# connection add/remove rates
conn_add_prob           = 0.5
conn_delete_prob        = 0.5

# connection enable options
enabled_default         = True
enabled_mutate_rate     = 0.01

feed_forward            = True
initial_connection      = full_direct

# node add/remove rates
node_add_prob           = 0.2
node_delete_prob        = 0.2

# network parameters
num_hidden              = 100
num_inputs              = 100
num_outputs             = 4

# node response options
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 30.0
response_min_value      = -30.0
response_mutate_power   = 0.0
response_mutate_rate    = 0.0
response_replace_rate   = 0.0

# connection weight options
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 30
weight_min_value        = -30
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 20
species_elitism      = 2

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2


Overwriting tron.config


In [ ]:

def run(config):

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 300)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    #winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    
    #node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
    #visualize.draw_net(config, winner, True, node_names=node_names)
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

    p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    p.run(eval_genomes, 50)
    return p, stats

In [ ]:
# Load configuration.
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     "tron.config")

p, stats = run(config)


 ****** Running generation 0 ****** 

Population's average fitness: 144.50667 stdev: 1147.96355
Best fitness: 10021.00000 - size: (54, 5600) - species 83 - id 83
Average adjusted fitness: 0.014
Mean genetic distance 3.765, standard deviation 0.285
Population of 324 members in 150 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     2     12.0    0.001     0
     2    0     2      3.0    0.000     0
     3    0     2     18.0    0.002     0
     4    0     2      2.0    0.000     0
     5    0     2      3.0    0.000     0
     6    0     2     25.0    0.002     0
     7    0     2     14.0    0.001     0
     8    0     2      9.0    0.001     0
     9    0     2     17.0    0.002     0
    10    0     2      1.0    0.000     0
    11    0     2     14.0    0.001     0
    12    0     2      2.0    0.000     0
    13    0     2     12.0    0.001     0
    14    0     2     18.0    0.002     0
    15    0     2      1.0    0.000  


 ****** Running generation 2 ****** 

Population's average fitness: 165.46769 stdev: 1231.20783
Best fitness: 10021.00000 - size: (54, 5570) - species 115 - id 235
Average adjusted fitness: 0.018
Mean genetic distance 3.764, standard deviation 0.285
Population of 323 members in 150 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    2     2     12.0    0.001     2
     2    2     2      3.0    0.000     2
     3    2     2     18.0    0.002     1
     4    2     2     11.0    0.001     0
     5    2     2      9.0    0.000     0
     6    2     2     21.0    0.002     2
     7    2     2     16.0    0.001     1
     8    2     2     14.0    0.001     1
     9    2     2     22.0    0.002     1
    10    2     2      7.0    0.000     1
    11    2     2     15.0    0.001     0
    12    2     2      2.0    0.000     1
    13    2     2     12.0    0.001     2
    14    2     2     15.0    0.001     2
    15    2     2     15.0    0.001


 ****** Running generation 4 ****** 

Population's average fitness: 137.13522 stdev: 1116.11119
Best fitness: 10028.00000 - size: (54, 5600) - species 59 - id 59
Average adjusted fitness: 0.010
Mean genetic distance 3.764, standard deviation 0.285
Population of 322 members in 150 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    4     2     12.0    0.001     4
     2    4     2      2.0    0.000     4
     3    4     2     19.0    0.002     3
     4    4     2     10.0    0.001     2
     5    4     2      6.0    0.000     1
     6    4     2     16.0    0.002     1
     7    4     2     17.0    0.002     3
     8    4     2     14.0    0.001     3
     9    4     2     20.0    0.002     3
    10    4     2      3.0    0.000     3
    11    4     2     11.0    0.001     1
    12    4     2      2.0    0.000     3
    13    4     2     12.0    0.001     4
    14    4     2     21.0    0.001     0
    15    4     2     16.0    0.001  


 ****** Running generation 6 ****** 

Population's average fitness: 166.89752 stdev: 1237.31830
Best fitness: 10029.00000 - size: (54, 5600) - species 146 - id 146
Average adjusted fitness: 0.018
Mean genetic distance 3.764, standard deviation 0.285
Population of 322 members in 150 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    6     2     11.0    0.001     6
     2    6     2      3.0    0.000     6
     3    6     2     21.0    0.002     0
     4    6     2      8.0    0.000     4
     5    6     2     11.0    0.001     0
     6    6     2     23.0    0.002     3
     7    6     2     20.0    0.002     5
     8    6     2     14.0    0.001     5
     9    6     2     18.0    0.002     5
    10    6     2      3.0    0.000     5
    11    6     2     16.0    0.001     3
    12    6     2      3.0    0.000     5
    13    6     2     12.0    0.001     6
    14    6     2     17.0    0.001     2
    15    6     2     11.0    0.001


 ****** Running generation 8 ****** 

Population's average fitness: 74.32919 stdev: 786.02500
Best fitness: 10017.00000 - size: (54, 5570) - species 136 - id 244
Average adjusted fitness: 0.008
Mean genetic distance 3.765, standard deviation 0.283
Population of 324 members in 150 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    8     2     12.0    0.001     8
     2    8     2      3.0    0.000     8
     3    8     2     22.0    0.002     0
     4    8     2     15.0    0.001     0
     5    8     2      8.0    0.001     2
     6    8     2     22.0    0.002     5
     7    8     2     19.0    0.002     7
     8    8     2     12.0    0.001     1
     9    8     2     19.0    0.002     7
    10    8     2      3.0    0.000     7
    11    8     2     14.0    0.001     5
    12    8     2      3.0    0.000     7
    13    8     2     12.0    0.001     8
    14    8     2     10.0    0.001     4
    15    8     2     15.0    0.001  

...

Population's average fitness: 267.54717 stdev: 1565.80081
Best fitness: 10020.00000 - size: (54, 5377) - species 60 - id 1652

Species 137 with 2 members is stagnated: removing it

Species 75 with 2 members is stagnated: removing it

Species 125 with 2 members is stagnated: removing it

Species 18 with 6 members is stagnated: removing it
Average adjusted fitness: 0.045
Mean genetic distance 3.685, standard deviation 0.541
Population of 160 members in 35 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     3   40     2     20.0    0.001    15
     5   40    21  10012.0    0.501    18
     6   40     3     21.0    0.002     3
     7   40     3     23.0    0.002     3
     9   40     2     25.0    0.002    13
    14   40     2     11.0    0.001    11
    16   40     2     26.0    0.002    19
    25   40    21  10007.0    0.499     0
    37   40     2     19.0    0.001     8
    39   40     2     24.0    0.002     9
    42   40     2     20.0   

Population's average fitness: 148.28000 stdev: 1146.93887
Best fitness: 10021.00000 - size: (54, 5487) - species 3 - id 2189

Species 86 with 2 members is stagnated: removing it

Species 135 with 2 members is stagnated: removing it

Species 133 with 2 members is stagnated: removing it
Average adjusted fitness: 0.011
Mean genetic distance 3.575, standard deviation 0.734
Population of 151 members in 22 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     3   45    29  10021.0    0.023     1
     6   45     2     24.0    0.002     8
     7   45     2     22.0    0.002     8
     9   45     2     25.0    0.002    18
    14   45     2     12.0    0.001    16
    25   45     2     18.0    0.001     5
    37   45     2     12.0    0.001    13
    39   45     2     24.0    0.002    14
    55   45     2     22.0    0.001    12
    70   45     2     23.0    0.002    12
    80   45     2     16.0    0.001    15
    81   45     2     26.0    0.002    17

Mean genetic distance 3.498, standard deviation 0.839
Population of 151 members in 15 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     3   51     2     18.0    0.002     7
     6   51     2     22.0    0.002    14
     7   51     2     20.0    0.002    14
    25   51     2     19.0    0.002    11
    37   51     2     21.0    0.002    19
    55   51    40  10024.0    0.057    18
    70   51    36  10027.0    0.106     0
    85   51     2     19.0    0.002     7
    87   51    25  10013.0    0.072     3
    97   51    24  10029.0    0.032     0
   108   51     5     23.0    0.001    11
   127   51     2     21.0    0.001    17
   130   51     2     15.0    0.001    16
   131   51     2     20.0    0.002    17
   132   51     3     32.0    0.002     5
Total extinctions: 0
Generation time: 28.729 sec (30.669 average)

 ****** Running generation 52 ****** 

Population's average fitness: 147.40397 stdev: 1144.23655
Best fitness: 10025.00000 -

Population's average fitness: 149.74172 stdev: 1143.61231
Best fitness: 10022.00000 - size: (53, 4933) - species 97 - id 4007

Species 25 with 2 members is stagnated: removing it

Species 108 with 2 members is stagnated: removing it
Average adjusted fitness: 0.010
Mean genetic distance 3.236, standard deviation 1.082
Population of 150 members in 6 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     3   60     3     19.0    0.002    16
    70   60    24     32.0    0.002     2
    85   60     3     15.0    0.001    16
    87   60    53  10019.0    0.024     2
    97   60    56  10022.0    0.029     9
   132   60    11     27.0    0.002    14
Total extinctions: 0
Generation time: 25.433 sec (26.882 average)

 ****** Running generation 61 ****** 

Population's average fitness: 417.60000 stdev: 1960.66942
Best fitness: 10033.00000 - size: (54, 4935) - species 97 - id 4149
Average adjusted fitness: 0.028
Mean genetic distance 3.333, standard dev

Mean genetic distance 3.200, standard deviation 1.080
Population of 149 members in 4 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     3   72    25     34.0    0.002     8
    70   72    32  10028.0    0.043    14
    87   72    51  10021.0    0.251    14
    97   72    41  10030.0    0.085     4
Total extinctions: 0
Generation time: 25.578 sec (3311.001 average)

 ****** Running generation 73 ****** 

Population's average fitness: 352.05369 stdev: 1801.60246
Best fitness: 10022.00000 - size: (53, 4752) - species 3 - id 5714
Average adjusted fitness: 0.040
Mean genetic distance 3.160, standard deviation 1.069
Population of 151 members in 4 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     3   73    51  10022.0    0.081     9
    70   73    17     34.0    0.001    15
    87   73    27     23.0    0.001    15
    97   73    56  10021.0    0.075     5
Total extinctions: 0
Generation time: 26.09